In [1]:
!pip install unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 255.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 206.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 MB 120.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 125.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 168.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 220.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 140.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 152.4 MB/s eta 0:00:0000:0100:01
   ━━

In [51]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-3B-Instruct",
    load_in_4bit = False, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

==((====))==  Unsloth 2025.7.6: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    NVIDIA A100 80GB PCIe. Num GPUs = 1. Max memory: 79.254 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.17it/s]


In [52]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 32,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 32,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

In [59]:
import json

with open('/workspace/Qwen2.5-VL/qwen-vl-finetune/demo/sequences_session_20250720_165353.json', 'r') as f:
    dataset = json.load(f)

In [9]:
dataset[0]

{'image': 'session_20250720_165353/frames/frame_000003.png',
 'conversations': [{'from': 'human',
   'value': '<image>\nclicks to move e2 to e4'},
  {'from': 'gpt',
   'value': '<tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 679]}}</tool_call><tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 543]}}</tool_call>'}]}

In [60]:
def convert_to_conversation_chess(sample):
    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : sample['conversations'][0]['value']},
            {"type" : "image", "image" : sample["image"]} ]
        },
        { "role" : "gpt",
          "content" : [
            {"type" : "text",  "text"  : sample["conversations"][1]["value"]} ]
        },
    ]
    return { "messages" : conversation }

In [11]:
sample = dataset[0]
convert_to_conversation_chess(sample)

{'messages': [{'role': 'user',
   'content': [{'type': 'text', 'text': '<image>\nclicks to move e2 to e4'},
    {'type': 'image',
     'image': 'session_20250720_165353/frames/frame_000003.png'}]},
  {'role': 'gpt',
   'content': [{'type': 'text',
     'text': '<tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 679]}}</tool_call><tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 543]}}</tool_call>'}]}]}

In [61]:
converted_dataset_chess = [convert_to_conversation_chess(sample) for sample in dataset]

In [15]:
converted_dataset[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text', 'text': '<image>\nclicks to move e2 to e4'},
    {'type': 'image',
     'image': 'session_20250720_165353/frames/frame_000003.png'}]},
  {'role': 'gpt',
   'content': [{'type': 'text',
     'text': '<tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 679]}}</tool_call><tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 543]}}</tool_call>'}]}]}

In [54]:
dataset[2]

{'image': 'session_20250720_165353/frames/frame_000016.png',
 'conversations': [{'from': 'human',
   'value': '<image>\nclicks to move g1 to h3'},
  {'from': 'gpt', 'value': [[596, 743], [632, 616]]}]}

In [53]:
## BEFORE TRAINING ##

import os
from PIL import Image


FastVisionModel.for_inference(model) # Enable for inference!

image_path = dataset[2]["image"]

image = Image.open(os.path.join("/workspace/Qwen2.5-VL/", image_path))
instruction = dataset[2]['conversations'][0]['value']

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 0.01, min_p = 0.1)

To move the piece from g1 to h3, you need to click on the square where the piece is currently located (g1) and then click on the target square (h3). Here’s how you can do it:

1. Click on the square where the piece is currently located (g1).
2. Then, click on the square where you want to move the piece (h3).

This will move the piece from g1 to h3.<|im_end|>


In [55]:
dataset[2]

{'image': 'session_20250720_165353/frames/frame_000016.png',
 'conversations': [{'from': 'human',
   'value': '<image>\nclicks to move g1 to h3'},
  {'from': 'gpt', 'value': [[596, 743], [632, 616]]}]}

In [56]:
from datasets import load_dataset
dataset = load_dataset("unsloth/LaTeX_OCR", split = "train")

In [57]:
instruction = "Write the LaTeX representation for this image."

def convert_to_conversation(sample):
    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : sample["image"]} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : sample["text"]} ]
        },
    ]
    return { "messages" : conversation }

converted_dataset = [convert_to_conversation(sample) for sample in dataset]
converted_dataset[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Write the LaTeX representation for this image.'},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=160x40>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '{ \\frac { N } { M } } \\in { \\bf Z } , { \\frac { M } { P } } \\in { \\bf Z } , { \\frac { P } { Q } } \\in { \\bf Z }'}]}]}

In [73]:
converted_dataset_chess[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text', 'text': '<image>\nclicks to move e2 to e4'},
    {'type': 'image',
     'image': 'session_20250720_165353/frames/frame_000003.png'}]},
  {'role': 'gpt',
   'content': [{'type': 'text', 'text': [[505, 679], [505, 543]]}]}]}

In [63]:
converted_dataset_chess[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text', 'text': '<image>\nclicks to move e2 to e4'},
    {'type': 'image',
     'image': 'session_20250720_165353/frames/frame_000003.png'}]},
  {'role': 'gpt',
   'content': [{'type': 'text',
     'text': '<tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 679]}}</tool_call><tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 543]}}</tool_call>'}]}]}

In [70]:
converted_dataset_chess[0]['messages'][1]['content'][0]['text']

'<tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 679]}}</tool_call><tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 543]}}</tool_call>'

In [71]:
for data_item in converted_dataset_chess:
    value = data_item['messages'][1]['content'][0]['text']
    print(value)
    break

<tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 679]}}</tool_call><tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 543]}}</tool_call>


In [72]:
all_clicks = []
for data_item in converted_dataset_chess:
    value = data_item['messages'][1]['content'][0]['text']
    clicks = []
    for tool_call in value.split('</tool_call>'):
        if not tool_call: continue
        tool_call = tool_call.replace('<tool_call>', '')
        data = json.loads(tool_call)
        clicks.append(data['arguments']['coordinate'])
    data_item['messages'][1]['content'][0]['text'] = clicks
    all_clicks.append(clicks)
print(all_clicks)

[[[505, 679], [505, 543]], [[547, 745], [421, 547]], [[596, 743], [632, 616]], [[505, 747], [582, 746]]]


In [81]:
converted_dataset_chess[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text', 'text': '<image>\nclicks to move e2 to e4'},
    {'type': 'image',
     'image': 'session_20250720_165353/frames/frame_000003.png'}]},
  {'role': 'gpt',
   'content': [{'type': 'text', 'text': [[505, 679], [505, 543]]}]}]}

In [74]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!


import torch._dynamo.config

# Increase cache limits for vision model training
torch._dynamo.config.cache_size_limit = 256
torch._dynamo.config.accumulated_cache_size_limit = 512


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    train_dataset = converted_dataset_chess,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 80,
        # num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_seq_length = 2048,
    ),
)

Unsloth: Model does not have a default image size - using 512


In [75]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4 | Num Epochs = 80 | Total steps = 80
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 82,169,856 of 3,836,792,832 (2.14% trained)


Step,Training Loss
1,6.446100
2,6.448600
3,6.051200
4,5.190100
5,3.914000
6,2.896700
7,2.209500
8,1.760300
9,1.490100
10,1.210400


In [79]:
## AFTER TRAINING ##

import os
from PIL import Image

FastVisionModel.for_inference(model) # Enable for inference!
image_path = dataset[0]["image"]

image = Image.open(os.path.join("/workspace/Qwen2.5-VL/", image_path))
instruction = dataset[0]['conversations'][0]['value']
print("--------------------------------")
print(instruction)
print("--------------------------------")

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 0.01, min_p = 0.1)

--------------------------------
<image>
clicks to move e2 to e4
--------------------------------
[[539, 641], [587, 599]]<|im_end|>


In [83]:
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]},
    {"role": "gpt", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]


tokenizer.apply_chat_template(messages, add_generation_prompt = True)

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|><image>\nclicks to move e2 to e4<|im_end|>\n<|im_start|>gpt\n<|vision_start|><|image_pad|><|vision_end|><image>\nclicks to move e2 to e4<|im_end|>\n<|im_start|>assistant\n'

In [80]:
dataset[0]

{'image': 'session_20250720_165353/frames/frame_000003.png',
 'conversations': [{'from': 'human',
   'value': '<image>\nclicks to move e2 to e4'},
  {'from': 'gpt',
   'value': '<tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 679]}}</tool_call><tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [505, 543]}}</tool_call>'}]}

In [35]:
dataset[2]

{'image': 'session_20250720_165353/frames/frame_000016.png',
 'conversations': [{'from': 'human',
   'value': '<image>\nclicks to move g1 to h3'},
  {'from': 'gpt',
   'value': '<tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [596, 743]}}</tool_call><tool_call>{"name": "computer_use", "arguments": {"action": "left_click", "coordinate": [632, 616]}}</tool_call>'}]}

In [48]:
all_clicks = []
for data_item in dataset:
    value = data_item['conversations'][1]['value']
    clicks = []
    for tool_call in value.split('</tool_call>'):
        if not tool_call: continue
        tool_call = tool_call.replace('<tool_call>', '')
        data = json.loads(tool_call)
        clicks.append(data['arguments']['coordinate'])
    data_item['conversations'][1]['value'] = clicks
    all_clicks.append(clicks)
print(all_clicks)

[[[505, 679], [505, 543]], [[547, 745], [421, 547]], [[596, 743], [632, 616]], [[505, 747], [582, 746]]]


In [50]:
dataset[2]

{'image': 'session_20250720_165353/frames/frame_000016.png',
 'conversations': [{'from': 'human',
   'value': '<image>\nclicks to move g1 to h3'},
  {'from': 'gpt', 'value': [[596, 743], [632, 616]]}]}

In [ ]:
value = dataset[2]['conversations'][1]['value']
import json
clicks = []
for tool_call in value.split('</tool_call>'):
    if not tool_call: continue
    tool_call = tool_call.replace('<tool_call>', '')
    data = json.loads(tool_call)
    clicks.append(data['arguments']['coordinate'])
print(clicks)

In [78]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory, 3)
# used_percentage = round(used_memory / max_memory * 100, 3)
# lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
# print(f"Peak reserved memory % of max memory = {used_percentage} %.")
# print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

124.0813 seconds used for training.
2.07 minutes used for training.
Peak reserved memory = 21.729 GB.
Peak reserved memory for training = 21.729 GB.
